# Part 2

Import Module

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

/Users/tengfone/.pyenv/versions/3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Pathings

In [2]:
TF_train_pathing = "/Users/tengfone/Downloads/train"
TF_test_pathing = "/Users/tengfone/Downloads/dev.in"

Loading Files

In [3]:
def open_train_file(file_path):
    with open(file_path) as f_lines:
        all_datas = f_lines.read().splitlines()
        all_datas[:] = [x for x in all_datas if x]

    word_tags = list()
    output_word_tags = list()
    for i in all_datas:
        i = i.split(' ')
        word_tags.append(i)
        
    for i in word_tags:
        if(len(i) == 3):
            i = [i[0]+i[1],i[2]]
            output_word_tags.append(i)
        else:
            output_word_tags.append(i)
    return output_word_tags

In [4]:
def open_test_file(file_path):
    with open(file_path) as f_lines:
        all_datas = f_lines.read().splitlines()

    output = list()
    para = 0

    for i in all_datas:
        if i != "":
            output.append([i, para])
        else:
            para += 1

    return output

Convert to DF

In [5]:
def to_train_df(word_tags):
    df = pd.DataFrame(word_tags, columns=['Text', 'Labels'])
    return df

In [6]:
def to_test_df(list_words):
    df = pd.DataFrame(list_words, columns=['Text', 'Paragraph'])
    return df

Emission Part a

In [7]:
def emission(train_data):
    # Total Number of Labels (DF)
    df_num_labels = train_data['Labels'].value_counts().rename_axis('Labels').reset_index(name='CountY')

    # Total Number of y -> x
    df_emission = train_data.groupby(["Text","Labels"]).size().reset_index()
    df_emission.columns = ["Text", "Labels", "CountY->X"]

    # e(x|y)
    emission_output = pd.merge(df_emission,df_num_labels, on = "Labels")
    emission_output["Emission"] = emission_output["CountY->X"]/emission_output["CountY"]

    return emission_output

Emission Part b (Fix with UNK)

In [8]:
def replace_unk(word,list_of_words):
    if(word not in list_of_words):
        return "#UNK#"
    else:
        return word

In [9]:
def emission_fix(train_data,test_data,k):
    list_of_words = train_data['Text'].to_list()
    test_data['Text'] = test_data['Text'].apply(lambda x: replace_unk(x,list_of_words))
  
  # Total Number of Labels (DF)
    df_num_labels = train_data['Labels'].value_counts().rename_axis('Labels').reset_index(name='CountY')

  # Total Number of y -> x
    df_emission = train_data.groupby(["Text","Labels"]).size().reset_index()
    df_emission.columns = ["Text", "Labels", "CountY->X"]

  # e(x|y)
    emission_output = pd.merge(df_emission,df_num_labels, on = "Labels")

  # Unique Labelsoutlist.append(output)
    unique_labels = emission_output.Labels.unique().tolist()
    print(emission_output)

    for i in range(len(unique_labels)):
        each_label = unique_labels[i]
        print(f"Running at {each_label}")
        count_y = emission_output[emission_output['Labels']==each_label]['CountY'].values[0]
        temp_list = list()
        for index, row in test_data.iterrows():
            word = row['Text']
            if (word == "#UNK#"):
                output = k/ (count_y + k)
                temp_list.append(output)
            else:
                count_yx = 0
                a = emission_output[(emission_output['Text'] == word)]
                a2 = a[(a['Labels'] == each_label)]
                # a = df.query('Text==@word and Labels==@label')
                if (a2.size==0):
                    output = 0
                    temp_list.append(output)
                else:
                    a2 = a2['CountY->X'].values[0]
                    count_yx = a2
                    output = count_yx/(count_y + k)
                    temp_list.append(output)
        test_data[each_label] = temp_list
                    
    return test_data

Get new Fixed Emission

In [10]:
k = 0.5
# Training
train_data = open_train_file(TF_train_pathing)
train_data_df = to_train_df(train_data)
emitted = emission(train_data_df)

# Testing
test_data = open_test_file(TF_test_pathing)

test_data_df = to_test_df(test_data)
emitted_fix = emission_fix(train_data_df,test_data_df,k)

                         Text      Labels  CountY->X  CountY
0                           !   B-neutral          1   24868
1                           "   B-neutral          4   24868
2                     #04-213   B-neutral          1   24868
3      #2017TaipeiUniversiade   B-neutral          1   24868
4                         #3D   B-neutral          1   24868
...                       ...         ...        ...     ...
50429                   youuu  B-negative          1    1139
50430                zenyatta  B-negative          2    1139
50431                   zhong  B-negative          1    1139
50432                    zouk  B-negative          1    1139
50433                   zubat  B-negative          1    1139

[50434 rows x 4 columns]
Running at B-neutral
Running at I-neutral
Running at I-positive
Running at O
Running at B-positive
Running at I-negative
Running at B-negative


Formatting Dataframe to Text/max_label/Paragraph

In [11]:
df2 = emitted_fix.drop(['Text', 'Paragraph'], axis=1)
df2['max_value'] = df2.max(axis=1)
df2['max_label'] = df2.idxmax(axis=1)
df3 = emitted_fix[['Text']].copy()
df3['Paragraph'] = emitted_fix[['Paragraph']].copy()
df3['max_label'] = df2[['max_label']].copy()
df3 = df3[['Text','max_label','Paragraph']]

In [12]:
print(df3)

             Text   max_label  Paragraph
0      Everything           O          0
1          sounds           O          0
2          better           O          0
3            with           O          0
4             the           O          0
...           ...         ...        ...
34186           @           O       2649
34187      Butter  B-positive       2649
34188          My           O       2649
34189       #UNK#  I-negative       2649
34190       #UNK#  I-negative       2649

[34191 rows x 3 columns]


Convert Dataframe to list with paragraphing

In [13]:
df3_idx = 0
output_list = list()
for index, row in df3.iterrows():
    word = row['Text']
    label = row['max_label']
    para = row['Paragraph']
    if(para == df3_idx):
        one_line = word + " " + label
        output_list.append([one_line])
    else:
        one_empty_line = ""
        output_list.append([one_empty_line])
        one_line = word + " " + label
        output_list.append([one_line])
        df3_idx += 1

Write to output file

In [14]:
with open('dev.prediction','w') as f:
    for item in output_list:
        for i in item:
            f.write("%s\n" % i)